# Money Flow: COT Positioning Deep-Dive

This notebook is a community-friendly reference workflow for FinUties COT analysis.

## What this notebook teaches
- How to load API credentials from a centralized `notebooks/.env`
- How to fetch latest COT data from FinUties
- How to derive net positioning and rolling z-score signals
- How to visualize positioning regime pressure with clear thresholds

## Inputs required
- `FINUTIES_API_KEY` in `notebooks/.env`
- Optional: `FINUTIES_API_ORIGIN` (defaults to `https://data.finuties.com`)

In [ ]:
from pathlib import Path
import os

import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns
from dotenv import load_dotenv

sns.set_theme(style="whitegrid")

In [ ]:
def resolve_notebooks_env(start_dir: Path) -> Path:
    current = start_dir.resolve()
    for candidate_root in [current, *current.parents]:
        if candidate_root.name == "notebooks":
            env_path = candidate_root / ".env"
            if env_path.exists():
                return env_path
        nested_env = candidate_root / "notebooks" / ".env"
        if nested_env.exists():
            return nested_env
    raise FileNotFoundError(
        "Missing notebooks/.env. Create it from notebooks/.env.example and set FINUTIES_API_KEY."
    )

env_path = resolve_notebooks_env(Path.cwd())
load_dotenv(env_path)

API_ORIGIN = os.getenv("FINUTIES_API_ORIGIN", "https://data.finuties.com").rstrip("/")
API_KEY = os.getenv("FINUTIES_API_KEY", "").strip()

if not API_KEY:
    raise ValueError("Missing FINUTIES_API_KEY. Create notebooks/.env from notebooks/.env.example and set your key.")

if "finuties.com" not in API_ORIGIN:
    print(f"Warning: API origin override in use: {API_ORIGIN}")

HEADERS = {"Authorization": f"Bearer {API_KEY}"}
TIMEOUT_SECONDS = 30

In [ ]:
COT_ENDPOINT = "/api/v1/cftc/legacy_futures-facts"
DEFAULT_LIMIT = 500


def finuties_get(endpoint: str, params: dict | None = None) -> dict | list:
    params = params or {}
    response = requests.get(
        f"{API_ORIGIN}{endpoint}",
        headers=HEADERS,
        params=params,
        timeout=TIMEOUT_SECONDS,
    )
    response.raise_for_status()
    return response.json()


def normalize_rows(payload: dict | list) -> list[dict]:
    if isinstance(payload, list):
        return payload
    return payload.get("items") or payload.get("data") or []

In [ ]:
payload = finuties_get(COT_ENDPOINT, {"limit": DEFAULT_LIMIT})
rows = normalize_rows(payload)

if not rows:
    raise ValueError("No COT rows returned from FinUties API.")

df = pd.DataFrame(rows)
required_cols = [
    "commodity_name",
    "report_date_as_yyyy_mm_dd",
    "noncomm_positions_long_all",
    "noncomm_positions_short_all",
]
missing_cols = [c for c in required_cols if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing expected COT columns: {missing_cols}")

df["report_date"] = pd.to_datetime(df["report_date_as_yyyy_mm_dd"], errors="coerce")
df["long"] = pd.to_numeric(df["noncomm_positions_long_all"], errors="coerce")
df["short"] = pd.to_numeric(df["noncomm_positions_short_all"], errors="coerce")
df = df.dropna(subset=["report_date", "long", "short"]).copy()
df["net_positioning"] = df["long"] - df["short"]

print(f"Rows loaded: {len(df):,}")
print("Available commodities:", df["commodity_name"].nunique())

In [ ]:
target_market = df["commodity_name"].value_counts().idxmax()
window = 26

series = (
    df[df["commodity_name"] == target_market]
    .sort_values("report_date")
    .reset_index(drop=True)
    .copy()
)

roll_mean = series["net_positioning"].rolling(window=window, min_periods=8).mean()
roll_std = series["net_positioning"].rolling(window=window, min_periods=8).std()
series["zscore_26w"] = ((series["net_positioning"] - roll_mean) / roll_std.replace(0, pd.NA)).fillna(0)
series["signal_band"] = pd.cut(
    series["zscore_26w"],
    bins=[-10, -2, 2, 10],
    labels=["short_extreme", "neutral", "long_extreme"],
)

series[["report_date", "commodity_name", "net_positioning", "zscore_26w", "signal_band"]].tail(12)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 6), sharex=True)

axes[0].plot(series["report_date"], series["net_positioning"], color="#1f77b4", linewidth=1.8, label="Net positioning")
axes[0].set_title(f"COT Net Positioning - {target_market}")
axes[0].set_ylabel("Contracts")
axes[0].legend(loc="upper left")

axes[1].plot(series["report_date"], series["zscore_26w"], color="#ff7f0e", linewidth=1.8, label="26-week z-score")
axes[1].axhline(2, linestyle="--", linewidth=1, color="#444444", label="Upper pressure")
axes[1].axhline(-2, linestyle="--", linewidth=1, color="#777777", label="Lower pressure")
axes[1].set_ylabel("Z-score")
axes[1].set_xlabel("Report date")
axes[1].legend(loc="upper left")

plt.tight_layout()
plt.show()

## Interpretation and caveats

- A high positive z-score indicates positioning is elevated versus its own 26-week history.
- A high negative z-score indicates positioning is unusually weak versus recent history.
- This is a distribution-relative signal, not a standalone directional forecast.
- Always validate with macro context, seasonality, and liquidity conditions.

## Community extension ideas

- Compare multiple commodities in one panel with standardized z-scores.
- Add a rolling percentile signal alongside z-score for robustness.
- Export transformed output for reuse in terminal plugin cards.

Reproducibility note: API content can change with new reports, revisions, and schema updates.